In [1]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
import Generate_word2vec_features
import Generate_bow_features
import Generate_tfidf_features
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import pandas as pd
import gensim
import os
import sys
import RunModels
sys.path.append( os.path.join(".."))
from UtilityFunctions import CommonHelpers,PreprocessHelpers,FeatureEngineering
import numpy as np
import pandas as pd
import re
from sklearn.metrics import f1_score

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sharath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sharath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Constants
book_pickle_path="../data/preprocessed/books_english.pickle"
name_pickle_path = "../data/preprocessed/names_english.pickle"

In [3]:
def get_number_id(file_name):
    regex = r"([a-z_A-Z]+)(\d+)"
    match = re.search(regex, file_name)
    if match!=None:
        return match.group(2)

In [4]:

book_names =  CommonHelpers.load_pickle(name_pickle_path)

In [6]:
list_of_words = CommonHelpers.load_pickle(book_pickle_path)

In [5]:
df = pd.read_csv("../data/samples_80_limit.csv")
df["category"] = df["LoCC"].str[:1]
category_list_train = [int(get_number_id(each)) for each in book_names]
selected_books = df[df["Text#"].isin(category_list_train)]
categories=[]
titles=[]
for each in category_list_train:
    categories.append(selected_books[selected_books["Text#"]==each]["category"].values[0])
    titles.append(selected_books[selected_books["Text#"]==each]["Title"].values[0])

# Different Word embeddings

# BOW

In [9]:
# BoW
# load the final vectors of size 1400*X (vary X ) into variable train_vec
# BoW book vectors (full length)
gen = Generate_bow_features.BoW_Vectors()
gen.load_pickles()
train_vec =gen.book_vectors

In [8]:
# BoW book vectors (SVD reduced dimension)
gen = Generate_bow_features.BoW_Vectors()
gen.load_pickles('SVD')
train_vec =gen.book_vectors

# TF-IDF

In [6]:
gen = Generate_tfidf_features.TFIDF_Vectors()
gen.load_unigram_pickles('SVD')
train_vec =gen.svd_unigram_tfidf_vectors

 __init__done
unigrams vector loaded


# Google Word2Vec model

In [ ]:
# Google Word2Vec book vectors
gen = Generate_word2vec_features.Word2Vec_Features()
gen.load_data(cache=True)
train_vec =np.nan_to_num( gen.book_vectors)



# Word2Vec model trained on LoCC books

In [ ]:
# Custom Word2Vec
# load the final vectors of size 1400*X (vary X ) into variable train_vec

gen = Generate_word2vec_features.Word2Vec_Features()

gen.load_data(model_type="self",cache=True)
train_vec =np.nan_to_num(gen.book_vectors)


# Run below cells to train and test the models

In [7]:
split_obj = RunModels.Split_Data(train_vec,categories)
split_obj.get_split()

In [ ]:
names = [
    "Nearest Neighbors",
    "RBF SVM",
    "Random Forest",
    "Naive Bayes",
    "Logistic Regression",
    "Logistic Regression One-vs-Rest",
    "LinearSVC One-vs-Rest",
    "Neural Net"
]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo'),
    RandomForestClassifier(n_estimators=400),
    GaussianNB(),
    LogisticRegression(multi_class='multinomial', random_state=0),
    LogisticRegression(multi_class='ovr', random_state=42, max_iter=500),
    OneVsRestClassifier(SVC()),
    MLPClassifier(alpha=0.05, max_iter=10000),
]

In [ ]:
accuracy_dict={}
f1_dict={}

In [ ]:
for name, clf in zip(names, classifiers):
    model = RunModels.Run_Model(clf,split_obj)
    model.run_model()
    result = model.get_predictions()
    metrics = model.get_metrics()
    accuracy_dict[name]=metrics[1]
    f1_dict[name]=metrics[2]
    print(f"{name}: Accuracy = {metrics[1]}, f1 score= {metrics[2]}")

In [ ]:
for key in accuracy_dict:
    print(key,",",accuracy_dict[key],",",f1_dict[key])

In [ ]:
f1_df

In [ ]:
import seaborn as sns
cf_matrix = metrics[0]
categories1 = set(categories)
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot = True, cmap = 'Blues',fmt = ''
,xticklabels = categories1, yticklabels = categories1)

In [ ]:
set(categories)